### TIFU KNN
В этом ноутбуке рассматривается один из самых эффективных алгоритмов для решения задачи next basket recommendation.  
Здесь используется его вариация, так как некоторые шаги могут быть заменены схожими по смыслу но другими по реализации.  

Примерный алгоритм следующий:
* Каждую корзину кодируем как вектор, где на i-й позиции стоит количество i-го товара в этой корзине
* Далее складываем все корзины каждого пользователя. Это можно сделать несколькими способами:
    - Как взвешенное среднее по всем корзинам
    - Как сумму векторов, умноженных на веса, позволяющих учитывать более старые корзины в меньшей степени (коэффициент **`r`** )  
* По полученным векторам находим ближайших соседей (параметр **`k_nearest`** ) каждого пользователя и далее можем их учитывать тоже несколькими способами:
    - Итоговый вектор пользователя - взвешенное среднее по всем соседям и самому пользователю - чем дальше сосед, тем меньше коэффициент (коэффициент **`alpha`**)
    - Итоговый вектор пользователя - `P = v_user * alpha + v_avg_neighbors * (1 - alpha)`, где `v_user` - вектор пользователя, `v_avg_neighbors` - средневзвешанное векторов соседей (коэффициент **`alpha`**)
    
* Далее находим позиции в итоговом векторе каждого пользователя, где значения максимальны - это и есть рекомендации (параметр **`top_k`** / **`nofix`**)
  
В оригинальном алгоритме корзины каждого пользователя разбиваются на группы, но так как в среднем покупок у пользователя не так много, этот шаг был убран

Оптимальной из проверенных комбинаций параметров оказалась:
* **`r = 0.75`**
* **`k_nearest = 30`**
* **`alpha = 0.95`**
* **`top_k = 18`**

Статья о TIFU KNN (2020): https://arxiv.org/pdf/2006.00556.pdf

! Ноутбук можно запускать в режиме "Run all"

## Загрузка библиотек

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.neighbors import NearestNeighbors
from implicit.lmf import LogisticMatrixFactorization
from scipy import sparse
from tqdm.auto import tqdm
tqdm.pandas()

## Подготовка данных

In [2]:
df = pd.read_csv("data/main.csv")

In [3]:
df.rename(columns={"order_completed_at":"time"}, inplace=True) # переименуем колонку для удобства
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # изменим тип колонки для дальнейшей работы

In [4]:
df.head()

,user_id,time,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [5]:
# Заменяет повторяющиеся товары в корзине на одну запись с указанием количества
def duplicates_to_count(t):
    return (
        t.groupby(['user_id', 'time'])['cart']
            .value_counts()
            .to_frame()
            .rename(columns={"cart":"count"})
            .reset_index()
    )

# Производит обратную операцию
def count_to_duplicates(t): 
    g = t.copy()
    g["to_explode"] = g["count"].apply(lambda x: [i for i in range(x)])
    g = g.explode("to_explode").drop(columns=["count", "to_explode"])
    return g

# Создает все комбинации user_id (пользователя) и cart (товара), которые пользователь когда-либо покупал
def make_skeleton(t):
    return t.groupby("user_id")["cart"].unique().to_frame().reset_index().explode("cart")

# Создает train датасет и ответы для него
# Ответы формируются на основе последней корзины каждого пользователя
# lvevel=1 означает, что для формирования ответов берётся последняя покупка, level=2 - предпоследняя и так далее 
def make_train_targets(t, level=1): 
    user_last_time = t.groupby(["user_id"])["time"].max().to_frame().reset_index()
    user_last_time["last_buy"] = 1
    
    train = pd.merge(t, user_last_time, on=["time", "user_id"], how="left")
    train = train[train["last_buy"] != 1]
    train.drop(columns=["last_buy"], inplace=True)
    
    if level >= 2:
        return make_train_targets(train, level-1)
    
    user_last_time.drop(columns=["last_buy"], inplace=True)
    
    user_last_carts = pd.merge(user_last_time, t, on=["user_id", "time"], how="left")
    
    skeleton = make_skeleton(t)
    targets = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")
    targets.fillna(0, inplace=True)
    targets["count"] = targets["count"].apply(lambda x: x if x <= 1 else 1).astype(int)
    targets.rename(columns={"count":"target"}, inplace=True)
    return train, targets

# Создает сводную таблицу покупок и товаров в них
def user_cart_pivot_table(t):
    g = pd.pivot_table(t, columns="cart", index=["user_id", "time"], values="count", aggfunc=np.sum, fill_value=0)
    for cart in list(set(df["cart"].unique()).difference(set(t["cart"].unique()))):
        g[cart] = 0
    return g

In [6]:
df = duplicates_to_count(df)

In [7]:
train, targets = make_train_targets(df, level=1)

## Применение подхода

### Параметры алгоритма

Установим исходные параметры алгоритма

In [8]:
r = 0.75
k_nearest = 30
alpha = 0.95 
top_k = 18

In [9]:
carts_count = 881
users_count = 20000

### Формирование векторов корзин

In [10]:
main = user_cart_pivot_table(train)

In [11]:
main = main.reindex(sorted(main.columns), axis=1).reset_index()

In [12]:
main.head()

cart,user_id,time,0,1,2,3,4,5,6,7,...,871,872,873,874,875,876,877,878,879,880
0,0,2020-07-19 09:59:17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2020-08-24 08:55:32,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2019-05-08 16:09:41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2020-01-17 14:44:23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2020-02-06 22:46:55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Отсортируем корзины по времени покупки для удобства дальнейшей обработки

In [13]:
main.sort_values('time', ascending=False, inplace=True)

In [14]:
main.head()

cart,user_id,time,0,1,2,3,4,5,6,7,...,871,872,873,874,875,876,877,878,879,880
134963,10497,2020-09-03 20:03:55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187074,19237,2020-09-03 19:42:35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139882,11107,2020-09-03 15:19:32,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
159623,13763,2020-09-03 14:35:33,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
161461,14041,2020-09-03 13:43:03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Представим в `users_all_carts` для каждого пользователя набор его корзин, закодированных как вектор

In [15]:
users_all_carts = [[] for i in range(users_count)]
main_array = main.to_numpy()
for arr in tqdm(main_array):
    users_all_carts[arr[0]].append(arr[2:])

  0%|          | 0/189406 [00:00<?, ?it/s]

In [16]:
users_all_carts[0][0][:30]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0], dtype=object)

### Формирование векторов пользователей

In [17]:
# Вектор пользователя формируется как среднее взвешенное всех его корзин (более старые корзины имеют меньший вес)
def get_users_one_cart_as_avg(users, r):
    users_as_avg_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        cur_weights = 0
        for j in range(len(users[i])):
            weight = r**(j)
            cart += pd.Series(users[i][j]) * weight
            cur_weights += weight
        if cur_weights == 0:
            cur_weights = 1
        cart /= cur_weights
        users_as_avg_cart[i] = cart.tolist()
    return users_as_avg_cart

# Вектор пользователя формируется как сумма векторов, умноженных на веса (более старые корзины имеют меньший вес)
def get_users_one_cart_as_attenuation(users, r):
    users_as_attenuation_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        for j in range(len(users[i])):
            cart += pd.Series(users[i][j]) * r**(j)
        users_as_attenuation_cart[i] = cart.tolist()
    return users_as_attenuation_cart

Применяем один из подходов для формирования пользователей в виде векторов из их корзин

In [18]:
users_as_vectors = get_users_one_cart_as_avg(users_all_carts, r)
# или
# users_as_vectors = get_users_one_cart_as_attenuation(users_all_carts, r)

  0%|          | 0/20000 [00:00<?, ?it/s]

### Объединение векторов пользователей

#### Поиск ближайших соседей

Ближайших соседей также можно искать несколькими способами:
* NearestNeighbors из sklearn
* Матричное разложение из implicit

In [19]:
def get_neighbors_nn(users_carts, k): # recommended
    model = NearestNeighbors()
    model.fit(users_carts)
    neighbors = model.kneighbors(users_carts, n_neighbors=k+1, return_distance=False)
    return neighbors

def get_neighbors_mf(users_carts, k):
    model = LogisticMatrixFactorization(iterations=100)
    model.fit(sparse.csr_matrix(users_carts).transpose())
    neighbors = [ [] for i in range(users_count)]
    for i in tqdm(range(users_count)):
        nbrs = model.similar_users(i, k+1)
        for tpl in nbrs:
            neighbors[i].append(tpl[0]) 
    return neighbors

Применим один из них для поиска похожих пользователей

In [20]:
neighbors = get_neighbors_nn(users_as_vectors, k_nearest) 
# или
# neighbors = get_neighbors_mf(users_as_vectors, k_nearest) 

#### Объединение векторов

In [21]:
# Итоговый вектор пользователя высчитывается по формуле P = v_user * alpha + v_avg_neighbors * (1 - alpha)
def combine_with_neighbors_avg(users_carts, neighbors, alpha):
    avg_nbrs = [ [] for i in range(users_count) ]
    for i in tqdm(range(users_count)):
        avg = pd.Series(np.zeros(carts_count))
        cur_weights = 0
        for j in range(1, len(neighbors[i])):
            avg += pd.Series(users_carts[neighbors[i][j]]) * (len(neighbors[i]) - j)
            cur_weights += (len(neighbors[i]) - j)
        avg /= (cur_weights if cur_weights != 0 else 1)
        avg_nbrs[i] = avg
    return [pd.Series(users_carts[i]) * alpha + pd.Series(avg_nbrs[i]) * (1-alpha) for i in range(users_count)]

# Итоговый вектор пользователя высчитывается как сумма вектора пользователя и векторов его соседей,
# умноженных на коэффициенты (чем дальше сосед, тем меньше коэффициент)
def combine_with_neighbors_attenuation(users_carts, neighbors, alpha): # recommended
    users_p = [ [] for i in range(users_count)]
    for i in tqdm(range(len(neighbors))):
        users_p[i] = pd.Series(users_carts[i])
        for j in range(1, k_nearest+1):
            users_p[i] += alpha**(j) * pd.Series(users_carts[neighbors[i][j]])
    return users_p

Применим один из способов объединения пользователя с похожими на него

In [22]:
# users_as_final_vectors = combine_with_neighbors_avg(users_as_vectors, neighbors, alpha)
# или
users_as_final_vectors = combine_with_neighbors_attenuation(users_as_vectors, neighbors, alpha)

  0%|          | 0/20000 [00:00<?, ?it/s]

### Формирование финальных рекомендаций

Сформируем итоговые рекомендации для каждого пользователя

#### Фиксированный размер корзины

In [23]:
users_best_basket = []
for i in tqdm(range(users_count)):
    users_best_basket.append(users_as_final_vectors[i].sort_values(ascending=False).head(top_k).index.tolist())

  0%|          | 0/20000 [00:00<?, ?it/s]

In [24]:
users_best_basket[:1]

[[57,
  14,
  84,
  22,
  82,
  383,
  409,
  432,
  379,
  382,
  61,
  5,
  430,
  23,
  41,
  398,
  441,
  402]]

#### Нефиксированный размер корзины

In [25]:
users_cart_len = (
    train.groupby(["user_id", "time"])["count"].sum()
    .reset_index()
    .groupby("user_id")["count"].median().round().astype(int)
    .reset_index()
    .rename(columns={"count": "cart_len"})
)["cart_len"].values

In [26]:
users_best_basket_nofix = []
for i in tqdm(range(users_count)):
    users_best_basket_nofix.append(
        users_as_final_vectors[i].sort_values(ascending=False).head(users_cart_len[i]).index.tolist()
    )

  0%|          | 0/20000 [00:00<?, ?it/s]

In [27]:
users_best_basket_nofix[:1]

[[57, 14, 84, 22, 82, 383, 409, 432, 379, 382, 61, 5, 430, 23, 41, 398]]

### Использование Apriori

Можно также добавлять к рекомендациям товары, найденные через закономерности в покупках (см. `2. Apriori.ipynb`)

In [28]:
add_carts_from_apriori = False

if add_carts_from_apriori:
    ap = pd.read_csv("data/apriori_top_20.csv")
    ap_map = {}
    for index, row in ap.iterrows():
        ap_map[int(row["from"])] = int(row["to"])
    
    for i in tqdm(range(users_count)):
        for key in ap_map.keys():
            if key in users_best_basket[i] and ap_map[key] not in users_best_basket[i]:
                users_best_basket[i].append(ap_map[key])

### Получение результатов

Сформируем датафрейм с рекомендациями

In [29]:
recs = pd.DataFrame()
recs["user_id"] = [i for i in range(users_count)]
recs["items"] = users_best_basket

In [30]:
recs.head()

,user_id,items
0,0,"[57, 14, 84, 22, 82, 383, 409, 432, 379, 382, ..."
1,1,"[55, 798, 169, 812, 171, 14, 170, 88, 198, 441..."
2,2,"[57, 61, 23, 382, 82, 84, 403, 398, 22, 17, 38..."
3,3,"[57, 61, 84, 398, 430, 382, 19, 41, 383, 22, 1..."
4,4,"[57, 398, 61, 84, 22, 420, 17, 14, 430, 388, 1..."


Приведём рекомендации в удобный для оценивания метрики вид

In [31]:
recs = recs.explode('items').rename(columns={'items': 'cart'})
recs['predict'] = 1

Добавим верные ответы

In [32]:
res = pd.merge(targets, recs, on=["user_id", 'cart'], how="left")

In [33]:
res['predict'] = res['predict'].fillna(0).astype(int)

In [34]:
res.head()

,user_id,cart,target,predict
0,0,14,0,1
1,0,20,0,0
2,0,57,1,1
3,0,82,0,1
4,0,379,0,1


In [35]:
round(f1_score(res["target"], res["predict"]), 5)

0.40996

Аналогично оценим вариант с нефиксированным размером корзины

In [36]:
recs = pd.DataFrame()
recs["user_id"] = [i for i in range(users_count)]
recs["items"] = users_best_basket_nofix
recs = recs.explode('items').rename(columns={'items': 'cart'})
recs['predict'] = 1
res = pd.merge(targets, recs, on=["user_id", 'cart'], how="left")
res['predict'] = res['predict'].fillna(0).astype(int)
round(f1_score(res["target"], res["predict"]), 5)

0.41169

#### Мини-исследование алгоритмов
При фиксированных параметрах 
* r = 0.8
* k_nearest = 20              
* alpha = 0.85    
* top_k = 15
* level = 1

Менялись варианты (1) усреднения корзин пользователя, (2) поиска ближайших соседей и (3) объединения пользователя с соседями. 
- (1)
    - avg - `get_users_one_cart_as_avg()`
    - att - `get_users_one_cart_as_attenuation()`
- (2)
    - nn - `get_neighbors_nn()`
    - mf - `get_neighbors_mf()`
- (3)
    - avg - `combine_with_neighbors_avg()`
    - att - `combine_with_neighbors_attenuation()`

Результаты получились следующие:
* (1)-(2)-(3) - score
* avg-nn-avg - 0.37851
* avg-nn-att - 0.39407 - best
* avg-mf-avg - 0.37861
* avg-mf-att - 0.38423
* att-nn-avg - 0.37727
* att-nn-att - 0.38949 
* att-mf-avg - 0.37817
* att-mf-att - 0.38835  
  
Что можно сказать точно - для усреднения корзин пользователя лучше использовать `get_users_one_cart_as_attenuation()`

In [37]:
# r = 0.75
# k_nearest = 30
# alpha = 0.95     Scores: 0.40996, 0.41104, 0.37132 - mean = 0.39744
# top_k = 18
# avg-nn-att

In [38]:
# r = 0.75
# k_nearest = 30
# alpha = 0.95     Scores: 0.41169, 0.40695, 0.36921 - mean = 0.39278
# nofix
# avg-nn-att

In [39]:
# r = 0.8
# k_nearest = 20             
# alpha = 0.85     Scores: 0.39407, 0.38983, 0.35291 - mean = 0.37894
# top_k = 15
# avg-nn-att

In [40]:
# r = 0.85
# k_nearest = 26    
# alpha = 0.91     Scores: 0.39005, 0.38579, 0.34770 - mean = 0.37451
# top_k = 14
# avg-nn-att

In [41]:
# r = 0.9
# k_nearest = 30    
# alpha = 0.75    Scores: 0.35904, 0.35214, 0.31353 - mean=0.34157
# top_k = 10
# avg-nn-att